# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo


Note: you may need to restart the kernel to use updated packages.


In [2]:
from zipfile import ZipFile
import pandas as pd
import json as json
import re
from pymongo import MongoClient
import warnings
warnings.filterwarnings('ignore')

In [3]:
client = MongoClient("mongodb://localhost:27017")

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
client.list_database_names()

['Companies', 'admin', 'config', 'local']

In [5]:
db = client.Companies 

In [6]:
colec = db.Companies

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [7]:
list(colec.find({'name':'Babelgum'},{'name': 1,'_id': 0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [8]:
list(colec.find({'number_of_employees': {'$gt': 5000}},
{'_id': 0,'name': 1,'number_of_employees': 1}).sort
('number_of_employees',1).limit(20))

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [9]:
list(colec.find({'$and': [{'founded_year': {'$lt': 2005}},
                 {'founded_year': {'$gt': 2000}}]},
                 {'name': True, 'founded_year': True, '_id': False}))

[{'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'SmugMug', 'founded_year': 2002},
 {'name': 'Skype', 'founded_year': 2003},
 {'name': 'Pando Networks', 'founded_year': 2004},
 {'name': 'Ikan', 'founded_year': 2003},
 {'name': 'delicious', 'founded_year': 2003},
 {'name': 'Topix', 'founded_year': 2002},
 {'name': 'Jobster', 'founded_year': 2004},
 {'name': 'AllPeers', 'founded_year': 2004},
 {'name': 'iContact', 'founded_year': 2003},
 {'name': 'blinkx', 'founded_year': 2004},
 {'name': 'Yelp', 'founded_year': 2004},
 {'name': 'Coghead', 'founded_year': 2003},
 {'name': 'Farecast', 'founded_year': 2003},
 {'name': 'KickApps', 'founded_ye

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [71]:
list(colec.find({'$and': [{'ipo.valuation_amount': {'$gt':100000000}}, {'founded_year': {'$lt':2010}}]},
               {'_id': False, 'name': True, 'ipo.valuation_amount': True}))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}},
 {'name': 'Jive Software', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Wix', 'ipo': {'valuation_amount': 750000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'Salesforce', 'ipo': {'

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [10]:
list(colec.find({'$and': [{'number_of_employees': {'$lt': 1000}},{'founded_year':{'$lt': 2005}}]},
{'_id': 0,'name': 1,'number_of_employees': 1}).sort
('number_of_employees',-1).limit(10))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Mozilla', 'number_of_employees': 800},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Yelp', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [15]:
list(colec.find({'partners': {'$ne':None}}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'permalink': 'abc2',
  'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
  'homepage_url': 'http://wetpaint-inc.com',
  'blog_url': 'http://digitalquarters.net/',
  'blog_feed_url': 'http://digitalquarters.net/feed/',
  'twitter_username': 'BachelrWetpaint',
  'category_code': 'web',
  'number_of_employees': 47,
  'founded_year': 2005,
  'founded_month': 10,
  'founded_day': 17,
  'deadpooled_year': 1,
  'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
  'alias_list': '',
  'email_address': 'info@wetpaint.com',
  'phone_number': '206.859.6300',
  'description': 'Technology Platform Company',
  'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
  'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
  'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and mon

### 7. All the companies that have a null type of value on the `category_code` field.

In [16]:
list(colec.find({'category_code': {'$type':"null"}},{'name': 1}).sort('number_of_employes', -1).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd67529827a'), 'name': 'MMDAYS'},
 {'_id': ObjectId('52cdef7c4bab8bd67529827b'), 'name': 'Inside Group'},
 {'_id': ObjectId('52cdef7c4bab8bd675298262'), 'name': 'VidKing'},
 {'_id': ObjectId('52cdef7c4bab8bd675298279'), 'name': 'Touch Clarity'},
 {'_id': ObjectId('52cdef7c4bab8bd67529826e'), 'name': 'Drigg'},
 {'_id': ObjectId('52cdef7c4bab8bd675298261'), 'name': 'Level9 Media'},
 {'_id': ObjectId('52cdef7c4bab8bd675298225'), 'name': 'Snimmer'},
 {'_id': ObjectId('52cdef7c4bab8bd6752980f6'), 'name': 'Collective'},
 {'_id': ObjectId('52cdef7c4bab8bd675298226'), 'name': 'KoolIM'},
 {'_id': ObjectId('52cdef7c4bab8bd675298276'), 'name': 'SpaceTime'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [19]:
list(colec.find({'$and': [{'number_of_employees': {'$gt': 99}},{'number_of_employees':{'$lt': 1000}}]},
{'_id': 0,'name': 1,'number_of_employees': 1}).sort
('number_of_employees',1).limit(10))

[{'name': 'Fon', 'number_of_employees': 100},
 {'name': 'INgage Networks', 'number_of_employees': 100},
 {'name': 'TrialPay', 'number_of_employees': 100},
 {'name': 'mig33', 'number_of_employees': 100},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'EchoSign', 'number_of_employees': 100},
 {'name': 'Verimatrix', 'number_of_employees': 100},
 {'name': 'Firetide', 'number_of_employees': 100},
 {'name': 'hi5', 'number_of_employees': 100},
 {'name': 'Redfin', 'number_of_employees': 100}]

### 9. Order all the companies by their IPO price in a descending order.

In [35]:
list(colec.find({'$and':[{'ipo':{'$ne': True}}]},
                          {'_id': 0, 'Name': 1,'ipo': 1}).sort('ipo',-1).limit(10))

[{'ipo': {'valuation_amount': 108960000000,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}},
 {'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'ipo': {'valuation_amount': 12800000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NASDAQ:GRPN'}},
 {'ipo': {'valuation_amount': 11000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2004,
   'pub_month': 6,
   'pub_day': 1

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [36]:
list(colec.find({'$and': [{'number_of_employees': {'$ne': True}}]},
{'_id': 0,'name': 1,'number_of_employees': 1}).sort
('number_of_employees',-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [40]:
list(colec.find({'$and': [{'founded_month': {'$gt':5 }}]},
                 {'name': True, 'founded_month': True, '_id': False}).sort('founded_month',1).limit(1000))

[{'name': 'Hyperrig', 'founded_month': 6},
 {'name': 'Meeta', 'founded_month': 6},
 {'name': 'Big Stage', 'founded_month': 6},
 {'name': 'IdeaFry', 'founded_month': 6},
 {'name': 'MeterNet', 'founded_month': 6},
 {'name': 'NEOS', 'founded_month': 6},
 {'name': 'Nsyght', 'founded_month': 6},
 {'name': 'Fishlabs', 'founded_month': 6},
 {'name': 'Epok', 'founded_month': 6},
 {'name': 'SocialGO', 'founded_month': 6},
 {'name': 'MetaSieve', 'founded_month': 6},
 {'name': 'snuzu', 'founded_month': 6},
 {'name': 'The Creators Vault', 'founded_month': 6},
 {'name': 'Wellwrittenwords', 'founded_month': 6},
 {'name': 'Perception System', 'founded_month': 6},
 {'name': 'SuperSaaS', 'founded_month': 6},
 {'name': 'Clutterme', 'founded_month': 6},
 {'name': 'FashionFreax GmbH', 'founded_month': 6},
 {'name': 'Telstra', 'founded_month': 6},
 {'name': 'PBworks', 'founded_month': 6},
 {'name': 'ConstructionDeal', 'founded_month': 6},
 {'name': 'pickrset', 'founded_month': 6},
 {'name': 'Model Metrics'

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [56]:
list(colec.find({'$and':[{'founded_year':{'$lt':2000}},{'acquisition.price_amount':{'$gt': 1e9}}]},
                {'_id': False,'name': True,'acquisition.price_amount': True,'founded_year': True}))

[{'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Sabre',
  'founded_year': 1960,
  'acquisition': {'price_amount': 4300000000}},
 {'name': 'Lastminute',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1008000000}},
 {'name': 'Digitas',
  'founded_year': 1980,
  'acquisition': {'price_amount': 1300000000}},
 {'name': 'Omniture',
  'founded_year': 1996,
  'acquisition': {'price_amount': 1800000000}},
 {'name': 'DoubleClick',
  'founded_year': 1996,
  'acquisition': {'price_amount': 1100000000}},
 {'name': 'BEA Systems',
  'founded_year': 1995,
  'acquisition': {'price_amount': 8500000000}},
 {'name': 'aQuantive',
  'founded_year': 1997,
  'acquisition': {'price_amount': 6400000000}},
 {'name': 'Responsys',
  'founded_year': 1998,
  'a

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [58]:
list(colec.find({'$and':[{'founded_year':{'$gt':2010}}]},
                {'_id': False,'name': True,'acquisition': True}).sort('acquisition.price_amount', 1))

[{'name': 'Mokitown', 'acquisition': None},
 {'name': 'CircleUp', 'acquisition': None},
 {'name': 'PeekYou', 'acquisition': None},
 {'name': 'headr', 'acquisition': None},
 {'name': 'Fixya', 'acquisition': None},
 {'name': 'Wamba', 'acquisition': None},
 {'name': 'Pinger', 'acquisition': None},
 {'name': 'Widgetbox', 'acquisition': None},
 {'name': 'RazorGator', 'acquisition': None},
 {'name': 'Advaliant', 'acquisition': None},
 {'name': 'Fluc', 'acquisition': None},
 {'name': 'Social Gaming Network',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': '',
   'source_description': '',
   'acquired_year': 2011,
   'acquired_month': 4,
   'acquired_day': 18,
   'acquiring_company': {'name': 'MindJolt', 'permalink': 'mindjolt'}}},
 {'name': 'Mobiluck', 'acquisition': None},
 {'name': 'Skydeck', 'acquisition': None},
 {'name': 'Simplicant', 'acquisition': None},
 {'name': 'Moblica', 'acquisition': None},
 {'name': 'Spr

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [86]:
list(colec.find({'founded_year':{'$ne': True}},{'_id':False,'name':True,'founded_year':True}).sort('founded_year', -1).limit(30)),

([{'name': 'Fixya', 'founded_year': 2013},
  {'name': 'iBazar', 'founded_year': 2013},
  {'name': 'Fluc', 'founded_year': 2013},
  {'name': 'Clowdy', 'founded_year': 2013},
  {'name': 'SEOGroup', 'founded_year': 2013},
  {'name': 'WhosCall', 'founded_year': 2013},
  {'name': 'Gimigo', 'founded_year': 2013},
  {'name': 'Tongxue', 'founded_year': 2013},
  {'name': 'Wamba', 'founded_year': 2013},
  {'name': 'VistaGen Therapeutics', 'founded_year': 2013},
  {'name': 'Advaliant', 'founded_year': 2013},
  {'name': 'Shopseen', 'founded_year': 2013},
  {'name': 'Pikk', 'founded_year': 2013},
  {'name': 'Carfeine', 'founded_year': 2012},
  {'name': 'PriceHub', 'founded_year': 2012},
  {'name': 'LoveAccess', 'founded_year': 2012},
  {'name': 'Ptch', 'founded_year': 2012},
  {'name': 'Widgetbox', 'founded_year': 2012},
  {'name': 'Skydeck', 'founded_year': 2012},
  {'name': 'Pinger', 'founded_year': 2012},
  {'name': 'Newstree', 'founded_year': 2012},
  {'name': 'Navara', 'founded_year': 2012},
 

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [68]:
list(colec.find({'$and':[{'founded_day':{'$lt':8}}]},
                {'_id': False,'name': True,'acquisition': True}).sort('acquisition.price_amount', -1).limit(10))

[{'name': 'Netscape',
  'acquisition': {'price_amount': 4200000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://news.cnet.com/2100-1023-218360.html',
   'source_description': 'AOL buys Netscape for $4.2 billion',
   'acquired_year': 1998,
   'acquired_month': 11,
   'acquired_day': 28,
   'acquiring_company': {'name': 'AOL', 'permalink': 'aol'}}},
 {'name': 'PayPal',
  'acquisition': {'price_amount': 1500000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://news.cnet.com/2100-1017-941964.html',
   'source_description': 'eBay picks up PayPal for $1.5 billion',
   'acquired_year': 2002,
   'acquired_month': 7,
   'acquired_day': 8,
   'acquiring_company': {'name': 'eBay', 'permalink': 'ebay'}}},
 {'name': 'Zappos',
  'acquisition': {'price_amount': 1200000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_url': 'http://techcrunch.com/2009/07/22/amazon-buys-zappos/',
   'source_description':

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [69]:
list(colec.find({'$and': [{'category_code': 'web'},{'number_of_employees':{'$gt': 4000}}]},
{'_id': 0,'name': 1,'number_of_employees': 1,'category_code': 1}).sort
('number_of_employees',1).limit(10))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [83]:
list(colec.find({'$and': [{'acquisition.price_amount': {'$gt': 1e4}},{'acquisition.price_currency_code': 'EUR'}]},
{'_id': 0,'name': 1,'acquisition.price_amount': 1,'price_currency_code': 1}).sort
('acquisition.price_currency_code',-1).limit(15))

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Mobile Trend', 'acquisition': {'price_amount': 7000000}},
 {'name': 'Citizenside', 'acquisition': {'price_amount': 30000}},
 {'name': 'Excite Europe', 'acquisition': {'price_amount': 2500000}},
 {'name': 'YelloYello', 'acquisition': {'price_amount': 2500000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}},
 {'name': 'Branded Payment Solutions',
  'acquisition': {'price_amount': 3100000}},
 {'name': 'Tuenti Technologies', 'acquisition': {'price_amount': 70000000}},
 {'name': 'BioMed Central', 'acquisition': {'price_amount': 43400000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [84]:
list(colec.find({'$and': [{'founded_month': {'$lt':5 }}]},
                 {'name': True, 'acquisition': True, '_id': False}).sort('founded_month',1).limit(10))

[{'name': 'Meetup', 'acquisition': None},
 {'name': 'CriticalMetrics', 'acquisition': None},
 {'name': 'ClipBlast!', 'acquisition': None},
 {'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Jajah',
  'acquisition': {'price_amount': 207000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.techcrunch.com/2009/12/23/confirmed-jajah-sold-207-million/',
   'source_description': 'Confirmed: Jajah Sold For $207 Million',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 20,
   'acquiring_company': {'name': 'O2', 'permalink': 'o2'}}},
 {'name': 'Steorn', 'acquisition': None},
 {'name': 'Wakoopa', 'acquisition': None},
 {'name': 'TripUp', 'acqu

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [91]:
list(colec.find({'$and': [{'founded_year': {'$lt': 2011}},
                 {'founded_year': {'$gt': 1999}},
                 {'acquisition.acquired_year': {'$ne': 2011}}]},
                 {'name': True, 'founded_year': True, '_id': False,'acquisition.acquired_year': True}
                ).sort('acquisition.acqured_year', 1).limit(10))

[{'name': 'Gizmoz',
  'founded_year': 2003,
  'acquisition': {'acquired_year': 2009}},
 {'name': 'Scribd', 'founded_year': 2007},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon',
  'founded_year': 2002,
  'acquisition': {'acquired_year': 2009}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Twitter', 'founded_year': 2006}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [105]:
list(colec.find({'deadpooled_year':{'$': 'founded_year'}},
                         {'_id': False,'name': True, 'deadpooled_year': True, 'founded_year': True}).limit(10))

#ESTE ES MI HUMILDE INTENTO, NO ENCUENTRO LA FORMA DE SACAR LA DIFERENCIA ENTRE 2 VALORES EN UN DICCIONARIO DE ESTA FORMA.

OperationFailure: unknown operator: $substract, full error: {'ok': 0.0, 'errmsg': 'unknown operator: $substract', 'code': 2, 'codeName': 'BadValue'}

In [109]:
def deadpool_after_thirdyear():
    query=colec.find({'deadpooled_year': {'$gt': 3 }},    
                            {'name': 1, '_id':0, 'deadpooled_year':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'deadpooled_year'])
    
    return df

In [110]:
deadpool_after_thirdyear().head() #Esto esta copiado de Alex para tener la solucion en mi jupyter y saber hacerlo de otra forma :)

,name,deadpooled_year
0,Omnidrive,2008
1,Babelgum,2013
2,Sparter,2008
3,Thoof,2013
4,Mercora,2008
